In [1]:
# General libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
# Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Import Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

## Preliminary work on the dataset

In [2]:
yahoo_df = pd.read_csv("../data/OIL.csv")
yahoo_df = yahoo_df.set_index("Date")


X = yahoo_df[['ROC_1', 'ROC_4', 'ROC_5', 'ROC_30', 'Return', '2 Day ROI', '3 Day ROI',
       '4 Day ROI', '30 Day ROI', 'RSI_1', 'RSI_2', 'RSI_3', 'RSI_4',
       'MACDs_12_26_9', 'SRSI_14', 'SRSI_30', 'CCI', 'Previous_differenced',
       'principalDf']]
y = yahoo_df["Up down"]

# Split in 80/20 the dataframeX
X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, test_size = 0.20,
                                                          shuffle=False)

# Split in 75/25 the remaining 80 %
X_train, X_valid, y_train, y_valid = train_test_split(X_train_80, y_train_80,
                                                      test_size = 0.25, shuffle=False)


# Normalize

In [3]:
def shift_norm(df):
    #df = data
    for column in df:
        df[column]=((df[column]-df[column].mean())/df[column].std())
    return df

In [4]:
X_train_80 = shift_norm(X_train_80)
X_test = shift_norm(X_test)

In [5]:
X_train = shift_norm(X_train)
X_valid = shift_norm(X_valid)

# Random Forest

In [6]:
# Fit the model
rfc = RandomForestClassifier(random_state = 42)
rfc.fit(X_train,y_train) 

# Predict
y_pred_train = rfc.predict(X_train)
y_pred_valid = rfc.predict(X_valid)

# Compute Accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

Train Accuracy: 1.00000 - Validation Accuracy: 0.52004


### Tuning parameters (VERY SLOW!)

RFC = RandomForestClassifier()

parameters = {  'max_depth': [10, 50, 70, 100, None],
                'max_features': ['auto', 'sqrt'],
                'min_samples_leaf': [1, 2, 4],
                'min_samples_split': [2, 5, 10],
                'n_estimators': [100, 500, 1000, 2000]       
             }

best_RGB = GridSearchCV(cv=2,estimator=RFC, 
                   param_grid=parameters)

best_RGB.fit(X_train, y_train)

print(best_RGB.best_estimator_)

In [7]:
# Fit
rfc = RandomForestClassifier(max_depth=100, max_features='auto', min_samples_leaf=4,
                       min_samples_split=10)

rfc.fit(X_train_80,y_train_80)

C:\Users\enric\anaconda3\envs\data_science\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=100, max_features='auto', min_samples_leaf=4,
                       min_samples_split=10)

In [8]:
# Predict
y_pred_test = rfc.predict(X_test)
y_prob = rfc.predict_proba(X_test)[:,1]
# Compute accuracy
test_acc = accuracy_score(y_test, y_pred_test)
print ("Test Accuracy: {:.5f}".format(test_acc) )

Test Accuracy: 0.51478


# Save accuracies

In [9]:

y_pred_test = rfc.predict(X_test)
y_prob = rfc.predict_proba(X_test)[:,1]
# Compute accuracy
test_acc = accuracy_score(y_test, y_pred_test)
print ("Test Accuracy: {:.5f}".format(test_acc) )

Test Accuracy: 0.51478


In [10]:
df_prediction = pd.read_csv("../data/model_accuracy_oil.csv", index_col = False)
df_prediction =  df_prediction.drop(columns=["Unnamed: 0"])



In [11]:
df_prediction["RF_pred"] = y_pred_test
df_prediction["RF_accuracy"] = [test_acc for x in range(len(y_pred_test))]
df_prediction["RF_prob"] = y_prob

df_prediction

,ANN_pred,ANN_accuracy,ANN_prob,y,kNN_pred,kNN_accuracy,kNN_prob,RF_pred,RF_accuracy,RF_prob
0,0,0.469018,0.404047,0,1,0.488084,1.0,0,0.514776,0.440374
1,0,0.469018,0.342436,1,0,0.488084,0.0,1,0.514776,0.558104
2,0,0.469018,0.339922,1,0,0.488084,0.0,1,0.514776,0.565385
3,0,0.469018,0.404326,0,1,0.488084,1.0,1,0.514776,0.522251
4,0,0.469018,0.361674,0,1,0.488084,1.0,1,0.514776,0.583792
...,...,...,...,...,...,...,...,...,...,...
1044,0,0.469018,0.448136,0,0,0.488084,0.5,0,0.514776,0.470678
1045,0,0.469018,0.448477,1,0,0.488084,0.5,0,0.514776,0.485184
1046,0,0.469018,0.438599,0,0,0.488084,0.5,0,0.514776,0.466790
1047,0,0.469018,0.417918,1,0,0.488084,0.0,1,0.514776,0.589712


In [12]:
df_prediction.to_csv("../data/model_accuracy_oil.csv")